<a href="https://colab.research.google.com/github/r73psv/univer/blob/master/ns_avdoshin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms

In [5]:
#  Загрузка набора MNIST
# преобразовываем изображение в тензоры, затем нормализуем
# путь куда грузим
DATA_PATH = './data'
# выполняемое преобразование над набором данных
trans = transforms.Compose([transforms.ToTensor(),
transforms.Normalize((0.1307,), (0.3081,))])
# грузим набор данных тренировочный
train_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=True,
transform=trans, download=True)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=64,
                                          shuffle=True, num_workers=2)
# грузим набор данных тестовый
test_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=False,
transform=trans)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=4,
                                         shuffle=False, num_workers=2)

100%|██████████| 9912422/9912422 [05:05<00:00, 32480.31it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 28881/28881 [00:00<00:00, 159801.61it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 1648877/1648877 [00:03<00:00, 454433.75it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 4542/4542 [00:00<00:00, 3091112.89it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [6]:
#задаем устройство на котором будет тенироваться сеть - видеопроцессор, если нет то процессор
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [7]:
#создаем свёрточную НС (4 свёрточных слоя и 2 линейных)
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,10,5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10,20, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(320, 50, bias=True)
        self.fc2 = nn.Linear(50, 10, bias=True)

    # реализация метода forward в классе Net
    def forward(self, input):
        x = self.pool1(F.sigmoid(self.conv1(input)))
        x = self.pool2(F.sigmoid(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

net = Net()
net.to(device)

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [13]:
##criterion = nn.L1Loss()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(4):
    running_loss = 0.0
    running_acc = 0.0
    for i,data in enumerate(trainloader,1):
        img,label = data
        #img = Variable(img)
        #label = Variable(label)
        out = net(img)
        #print(out)
        loss = criterion(out,label) #loss
        running_loss += loss.item() * label.size(0)
        #total loss
        _,pred = torch.max(out,1)
        num_correct = (pred == label).sum()
        #accuracy = (pred == label).float().mean()
        running_acc += num_correct.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Train{} epoch, Loss: {:.6f},Acc: {:.6f}'.format(epoch+1,running_loss / (len(train_dataset)),running_acc / (len(train_dataset))))

Train1 epoch, Loss: 0.083222,Acc: 0.974867
Train2 epoch, Loss: 0.055230,Acc: 0.982817
Train3 epoch, Loss: 0.046436,Acc: 0.985433
Train4 epoch, Loss: 0.037969,Acc: 0.988250


In [14]:
net.eval()
eval_loss = 0
eval_acc = 0
for data in testloader:
    img,label = data
    #img = Variable(img,volatile=True)
    out = net(img)
    loss = criterion(out,label)
    eval_loss += loss.item() * label.size(0)    #total loss
    _,pred = torch.max(out,1)
    num_correct = (pred == label).sum()
    eval_acc += num_correct.item()

print('Test Loss:{:.6f},Acc: {:.6f}'
      .format(eval_loss/ (len(test_dataset)),eval_acc * 1.0/(len(test_dataset))))

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Test Loss:0.050044,Acc: 0.983000
